# 0. Условие задачи

Разработать алгоритм восстановления пробелов в слитном тексте.

**Требования:**
- Решение должно **запускаться без ошибок** у проверяющего и быть **воспроизводимым**.
- Приоритеты: **точность**, **скорость**, **легковесность**.

# 1. Импорт библиотек и загрузка датасета

In [ ]:
import itertools
import json
import math
import random
import re
from collections import Counter, defaultdict
from functools import lru_cache
from typing import Iterable, List, Tuple


import numpy as np
import pandas as pd
from datasets import load_dataset
from sklearn.metrics import f1_score
from tqdm import tqdm
from transformers import AutoTokenizer

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, IterableDataset

In [ ]:
# пришлось загружать именно так, потому что в тексте есть запятые, из - за которых pandas выдает ошибку
data = []
with open('dataset_1937770_3.txt', 'r', encoding='utf-8') as f:

    header = f.readline().strip().split(',')

    for line in f:
        line = line.strip()

        first_comma_index = line.find(',')

        if first_comma_index != -1:

            row_id = line[:first_comma_index]
            text = line[first_comma_index + 1:]
            data.append([row_id, text])


task_data = pd.DataFrame(data, columns=['id', 'text_no_spaces'])

In [ ]:
task_data.head(5)

,id,text_no_spaces
0,0,куплюайфон14про
1,1,ищудомвПодмосковье
2,2,сдаюквартирусмебельюитехникой
3,3,новыйдивандоставканедорого
4,4,отдамдаромкошку


In [ ]:
!pip install transformers sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 18.8 MB/s eta 0:00:00


# 2. Обучение LSTM + алгоритм DP

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
DEBUG = False   # обучение в боевом режиме (True использовалось для проверки правильности кода)

SEED = 42
random.seed(SEED)

if DEBUG:
    # RuBQ_2.0 маленький → достаточно небольших лимитов
    MAX_DOCS_RUBQ_QA   = 10_000   # берём все (ограничение сверху)
    MAX_DOCS_PARAGRAPHS = 0       # 0 = не использовать paragraphs в отладке
    MAX_SENT_TOTAL      = 100_000
    WARMUP_SENT         = 10_000
    EPOCHS              = 3
    EMB_DIM             = 64
    HIDDEN              = 128
else:
    # «боевой» вариант: добавляем paragraphs (≈57k строк)
    MAX_DOCS_RUBQ_QA   = 10_000   # все вопросы/ответы из dev+test
    MAX_DOCS_PARAGRAPHS = 60_000  # подключаем абзацы
    MAX_SENT_TOTAL      = 1_000_000
    WARMUP_SENT         = 80_000
    EPOCHS              = 8
    EMB_DIM             = 128
    HIDDEN              = 256

BATCH_SIZE = 128
MAX_LEN    = 128
LR         = 1e-3
DEVICE     = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def stream_sources() -> Iterable[Tuple[str, str]]:
    """
    Поток текстов из:
      - d0rj/RuBQ_2.0  (splits: dev, test) → question_text + answer_text
      - d0rj/RuBQ_2.0-paragraphs (split: paragraphs) → paragraph/paragraphs/text
    Все загрузки в режиме streaming=True (без кэша на диск).
    """
    # 1) RuBQ QA (dev + test)
    rubq_dev  = load_dataset("d0rj/RuBQ_2.0", split="dev",  streaming=True)
    rubq_test = load_dataset("d0rj/RuBQ_2.0", split="test", streaming=True)

    def head_rows(ds, limit, fields=("question_text", "answer_text")):
        taken = 0
        for row in ds:
            for f in fields:
                txt = row.get(f)
                if isinstance(txt, str) and txt.strip():
                    yield txt
                    taken += 1
                    if taken >= limit:
                        return

    for txt in head_rows(rubq_dev,  MAX_DOCS_RUBQ_QA):
        yield ("rubq", txt)
    for txt in head_rows(rubq_test, MAX_DOCS_RUBQ_QA):
        yield ("rubq", txt)

    # 2)ВАЖНО: split="paragraphs"
    if MAX_DOCS_PARAGRAPHS > 0:
        rubq_par = load_dataset("d0rj/RuBQ_2.0-paragraphs", split="paragraphs", streaming=True)
        taken = 0
        for row in rubq_par:
            txt = row.get("paragraph") or row.get("paragraphs") or row.get("text")
            if isinstance(txt, str) and txt.strip():
                yield ("rubq_par", txt)
                taken += 1
                if taken >= MAX_DOCS_PARAGRAPHS:
                    break

# простая сегментация предложений
_sent_re = re.compile(r"[.!?]+")
def sentences_from_text(text: str) -> List[str]:
    text = text.replace("\n", " ").strip()
    sents = _sent_re.split(text)
    # RuBQ вопросы/ответы короткие → порог делаем ниже
    return [s.strip() for s in sents if len(s.strip()) > 5]


# 2) Строим алфавит (тёплый проход)
def build_alphabet() -> Tuple[dict, dict]:
    counter = Counter()
    seen = 0
    for _, raw in stream_sources():
        for s in sentences_from_text(raw):
            s = s.lower().replace("ё","е")
            counter.update(ch for ch in s if ch != " ")
            seen += 1
            if seen >= WARMUP_SENT:
                break
        if seen >= WARMUP_SENT:
            break
    char2id = {"<pad>":0, "<unk>":1}
    for i, ch in enumerate(sorted(counter), start=2):
        char2id[ch] = i
    id2char = {i:c for c,i in char2id.items()}
    print(f"[alphabet] unique chars: {len(char2id)} (from {seen} sentences)")
    return char2id, id2char

char2id, id2char = build_alphabet()


# 3) Формирование обучающих примеров
def make_xy(sentence: str, char2id: dict, max_len: int):
    s = sentence.lower().replace("ё","е")
    xs, ys = [], []
    prev_nonspace = False
    for ch in s:
        if ch == " ":
            if prev_nonspace and ys:
                ys[-1] = 1
            continue
        xs.append(char2id.get(ch, 1))
        ys.append(0)
        prev_nonspace = True
    if len(xs) <= 1:
        return None
    if len(xs) > max_len:
        xs = xs[:max_len]
        ys = ys[:max_len]
    ys = ys[:-1]
    if not ys:
        return None
    return torch.tensor(xs, dtype=torch.long), torch.tensor(ys, dtype=torch.float32)

# 4) IterableDataset (поток в модель)
class SegmIterable(IterableDataset):
    def __init__(self, char2id, max_len, max_sent_total, split_ratio, role):
        """
        split_ratio: доля train
        role: 'train' или 'val'
        """
        super().__init__()
        self.char2id = char2id
        self.max_len = max_len
        self.max_sent_total = max_sent_total
        self.split_ratio = split_ratio
        self.role = role

    def __iter__(self):
        rng = random.Random(SEED)
        sent_count = 0
        for _, raw in stream_sources():
            sents = sentences_from_text(raw)
            rng.shuffle(sents)
            for s in sents:
                # детерминированно раскидываем по сплитам
                h = hash(s) ^ SEED
                in_train = (h % 1000) < int(self.split_ratio * 1000)
                if (self.role == "train" and not in_train) or (self.role == "val" and in_train):
                    continue
                ex = make_xy(s, self.char2id, self.max_len)
                if ex is None:
                    continue
                yield ex
                sent_count += 1
                if sent_count >= self.max_sent_total:
                    return

def collate_pad(batch):
    xs, ys = zip(*batch)
    T = max(len(x) for x in xs)
    B = len(xs)
    x_pad = torch.zeros((B, T), dtype=torch.long)
    y_pad = torch.full((B, T-1), -100.0, dtype=torch.float32)
    lengths = []
    for i,(x,y) in enumerate(zip(xs,ys)):
        x_pad[i,:len(x)] = x
        y_pad[i,:len(y)] = y
        lengths.append(len(x))
    return x_pad, y_pad, torch.tensor(lengths, dtype=torch.long)

train_set = SegmIterable(char2id, MAX_LEN, int(MAX_SENT_TOTAL*0.9), split_ratio=0.9, role="train")
val_set = SegmIterable(char2id, MAX_LEN, int(MAX_SENT_TOTAL*0.1), split_ratio=0.9, role="val")

train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_pad)
val_loader = DataLoader(val_set,   batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_pad)

# 5) Модель BiLSTM
class CharBoundaryTagger(nn.Module):
    def __init__(self, vocab_size, emb_dim=EMB_DIM, hidden=HIDDEN):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        self.rnn = nn.LSTM(emb_dim, hidden, batch_first=True, bidirectional=True)
        self.out = nn.Linear(hidden*2, 1)
    def forward(self, x, lengths):
        emb = self.emb(x)  # [B,T,E]
        packed = nn.utils.rnn.pack_padded_sequence(emb, lengths.cpu(), batch_first=True, enforce_sorted=False)
        h,_ = self.rnn(packed)
        h,_ = nn.utils.rnn.pad_packed_sequence(h, batch_first=True)   # [B,T,H*2]
        logits = self.out(h).squeeze(-1)                              # [B,T]
        return logits[:, :-1]  # T-1 щелей

# 6) Обучение + валидация по F1
def evaluate(model, loader):
    model.eval()
    all_true, all_pred = [], []
    with torch.no_grad():
        for x,y,lengths in loader:
            x,y,lengths = x.to(DEVICE), y.to(DEVICE), lengths.to(DEVICE)
            logits = model(x, lengths)
            mask = (y != -100.0)
            probs = torch.sigmoid(logits)
            preds = (probs > 0.5).float()
            all_true.append(y[mask].cpu())
            all_pred.append(preds[mask].cpu())
    if not all_true:
        return 0.0
    y_true = torch.cat(all_true).numpy()
    y_pred = torch.cat(all_pred).numpy()
    return f1_score(y_true, y_pred)

model = CharBoundaryTagger(len(char2id)).to(DEVICE)
opt = torch.optim.Adam(model.parameters(), lr=LR)

for epoch in range(EPOCHS):
    model.train()
    pbar = tqdm(train_loader, desc=f"Epoch {epoch}")
    for x,y,lengths in pbar:
        x,y,lengths = x.to(DEVICE), y.to(DEVICE), lengths.to(DEVICE)
        logits = model(x, lengths)
        mask = (y != -100.0)
        loss = F.binary_cross_entropy_with_logits(logits[mask], y[mask])
        opt.zero_grad(); loss.backward(); opt.step()
        pbar.set_postfix(loss=float(loss))
    f1 = evaluate(model, val_loader)
    print(f"[epoch {epoch}] val F1 = {f1:.4f}")

# 7) Сохранение весов и алфавита
torch.save(model.state_dict(), "bilstm_ru_stream_no_oscar.pth")
with open("char2id_stream_no_oscar.json","w",encoding="utf8") as f:
    json.dump(char2id,f,ensure_ascii=False)
print("Saved: bilstm_ru_stream_no_oscar.pth, char2id_stream_no_oscar.json")


In [ ]:
# Автоконфиг загрузчик BiLSTM

class CharBoundaryTagger(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        self.rnn = nn.LSTM(emb_dim, hidden, batch_first=True, bidirectional=True)
        self.out = nn.Linear(hidden*2, 1)
    def forward(self, x, lengths):
        emb = self.emb(x)
        packed = nn.utils.rnn.pack_padded_sequence(
            emb, lengths.cpu(), batch_first=True, enforce_sorted=False
        )
        h,_ = self.rnn(packed)
        h,_ = nn.utils.rnn.pad_packed_sequence(h, batch_first=True)
        logits = self.out(h).squeeze(-1)
        return logits[:, :-1]

def load_bilstm_autoconfig(paths, device="cpu"):
    """
    paths = {"weights": "...pth", "char2id": "...json"}
    Автоматически определяет emb_dim и hidden из чекпойнта.
    """
    with open(paths["char2id"], "r", encoding="utf8") as f:
        char2id = json.load(f)

    sd = torch.load(paths["weights"], map_location="cpu")
    emb_dim = sd["emb.weight"].shape[1]
    out_in = sd["out.weight"].shape[1]
    hidden = out_in // 2

    model = CharBoundaryTagger(len(char2id), emb_dim=emb_dim, hidden=hidden)
    model.load_state_dict(sd, strict=True)
    model.to(device).eval()
    print(f"[BiLSTM] loaded emb_dim={emb_dim}, hidden={hidden}, vocab={len(char2id)}")
    return model, char2id

In [ ]:
# boundary_probs_for_string: считает p_j для щелей строки

MAX_LEN = 128

@torch.no_grad()
def boundary_probs_for_string(s: str, model, char2id, device="cpu"):
    s = s.lower().replace("ё", "е")
    if len(s) <= 1:
        return []
    def _one_pass(seq: str):
        x = torch.tensor([[char2id.get(ch, 1) for ch in seq]], dtype=torch.long, device=device)
        lengths = torch.tensor([x.size(1)], dtype=torch.long, device=device)
        logits = model(x, lengths)         # [1, T-1]
        return torch.sigmoid(logits).squeeze(0).tolist()
    if len(s) <= MAX_LEN:
        return _one_pass(s)
    # скользящее окно c перекрытием
    win, step = MAX_LEN, MAX_LEN - 16
    acc = [0.0]*(len(s)-1); cnt = [0]*(len(s)-1)
    start = 0
    while start < len(s):
        end = min(len(s), start+win)
        probs = _one_pass(s[start:end])
        for k,p in enumerate(probs):
            gi = start + k
            if gi < len(acc):
                acc[gi]+=p; cnt[gi]+=1
        if end == len(s): break
        start += step
    return [acc[i]/max(1,cnt[i]) for i in range(len(acc))]

In [ ]:
# score_word
def score_word(word):
    """
    Улучшенный скор слова:
    - Так как есть предобученный tokenizer в окружении, используем -len(subtokens),
      а UNK сильно штрафуем;
    """
    if not word:
        return -1e9  # сильный штраф пустой строке

    try:
        tok = tokenizer
    except NameError:
        tok = None

    if tok is not None:
        try:
            toks = tok.tokenize(word)
            if getattr(tok, "unk_token", None) in toks:
                return -50.0  # жёсткий штраф за UNK
            return -float(len(toks))  # меньше сабтокенов — лучше
        except Exception:
            pass  # на всякий случай — уходим во фолбэк ниже

    # Фолбэк без токенайзера: максимум около длины ~6
    L = len(word)
    return -abs(6 - L) * 0.1


# локальные помощники
_PUNCT = set(",.;:!?…()[]{}\"'—–-")

def _is_cyr(c: str) -> bool:
    c = c.lower()
    return ('а' <= c <= 'я') or (c == 'ё')

def _hard_boundary(prev_ch, ch) -> bool:
    """
    Подсказка, что между prev_ch и ch вероятна граница слова:
    - пунктуация
    - буква↔цифра
    - латиница↔кириллица (цифры игнорируем)
    - смена регистра: lower -> Upper
    """
    if prev_ch is None or ch is None:
        return False
    if ch in _PUNCT or prev_ch in _PUNCT:
        return True
    if prev_ch.isdigit() != ch.isdigit():
        return True
    if (_is_cyr(prev_ch) != _is_cyr(ch)) and (not prev_ch.isdigit() and not ch.isdigit()):
        return True
    if prev_ch.islower() and ch.isupper():
        return True
    return False

# Устойчивая интеграция DP + LSTM
assert '_PUNCT' in globals() and '_hard_boundary' in globals() and 'score_word' in globals(), \
    "Должны быть _PUNCT, _hard_boundary, score_word"

_SHORT_OK = {"в","к","с","и","у","я","о","а"}  # whitelist односимвольных

@lru_cache(maxsize=200_000)
def _cached_score_word(w: str) -> float:
    return score_word(w)

def restore_spaces_dp_lstm(
    text: str,
    model=None, char2id=None, device="cpu",
    alpha: float = 0.6,
    max_word_len: int = 28,
    hint_bonus: float = 0.15,
    use_logit: bool = True,
    temp: float = 1.4,
    bias: float = -0.2,
    conf_threshold: float = 0.6,
    lambda_cut: float = 0.08,
    short_penalty: float = 1.0
) -> str:
    if not isinstance(text, str) or len(text) == 0:
        return ""

    # 1) Разбивка на участки без пунктуации и сами знаки
    parts, i = [], 0
    while i < len(text):
        ch = text[i]
        if ch in _PUNCT:
            parts.append((ch, True)); i += 1 # сам знак препинания идёт отдельным токеном
        else:
            j = i + 1
            while j < len(text) and text[j] not in _PUNCT:
                j += 1
            parts.append((text[i:j], False)); i = j # непрерывный кусок без пунктуации

    tokens = []
    for chunk, is_punct in parts:
        if is_punct:
            tokens.append(chunk) # пунктуацию переносим «как есть»
            continue

        run = chunk
        n = len(run)
        if n == 0:
            continue

        # 2) эвристики - отмечаем позиции, где "вероятна" граница
        hints = [False]*(n+1) # hints[k] — граница перед символом с индексом k
        for k in range(1, n):
            if _hard_boundary(run[k-1], run[k]): hints[k] = True

        # 3) вероятности сети для этого run
        probs = None
        if (model is not None) and (char2id is not None) and (n > 1):
            probs = boundary_probs_for_string(run, model, char2id, device=device)  # len = n-1
            # probs[j-1] — вероятность разреза между run[j-1] и run[j]

        # 4) DP dp[end] — лучший суммарный скор для run[:end]; prv[end] — откуда пришли
        dp  = np.full(n+1, -1e18, dtype=float)
        prv = np.full(n+1, -1,   dtype=int)
        dp[0] = 0.0

        for end in range(1, n+1):
            start_lim = max(0, end - max_word_len)
            best_val, best_j = -1e18, -1

            for j in range(start_lim, end):
                cand = run[j:end]
                s = _cached_score_word(cand)

                # штраф за односимвольные вне whitelist
                if len(cand) == 1 and cand.lower() not in _SHORT_OK:
                    s -= short_penalty

                if j > 0 and hints[j]:
                    s += hint_bonus # бонус, если в позиции j эвристика «видит» границу

                # вклад нейросети (если уверенность >= порога)
                if (probs is not None) and (j > 0) and (j < n):
                    p = min(max(probs[j-1], 1e-6), 1 - 1e-6)
                    if p >= conf_threshold: # учитываем только уверенные предсказания
                        if use_logit:
                            logit = math.log(p) - math.log(1-p)
                            nn_term = (logit + bias) / max(1e-6, temp)
                        else:
                            logit = math.log(p) - math.log(1-p)
                            p_adj = 1 / (1 + math.exp(-(logit + bias)/max(1e-6, temp)))
                            nn_term = math.log(p_adj)
                        s += alpha * nn_term

                # штраф за сам факт разреза (регуляризация)
                if j > 0:
                    s -= lambda_cut

                val = dp[j] + s # суммарный скор до j + локальный скор текущего слова
                # тай-брейкер в пользу МЕНЬШЕ разрезов
                if (val > best_val) or (val == best_val and j == start_lim):
                    best_val, best_j = val, j

            dp[end], prv[end] = best_val, best_j # сохраняем лучший вариант для префикса [:end]

        # бэктрекинг: вытаскиваем слова из prv
        if prv[n] == -1:
            tokens.append(run)
        else:
            rev = []; k = n
            while k > 0:
                j = prv[k]; rev.append(run[j:k]); k = j
            tokens.extend(rev[::-1])

    # 5) Склейка с пунктуацией
    out = []
    for k, tok in enumerate(tokens):
        out.append(tok)
        if k == len(tokens) - 1:
            break
        nxt = tokens[k + 1]
        if tok in _PUNCT:
            if tok in {",","; ",":","!","?","…","."}:
                out.append(" ")
            else:
                out.append(" ")
        elif nxt in _PUNCT:
            pass
        else:
            out.append(" ")

    result = "".join(out).strip()
    # гарантия типа
    if not isinstance(result, str):
        result = str(result)
    return result

# 3. Проверка DP + LSTM на нашем датасете

In [ ]:
# Загрузка модели и проверка
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BILSTM_PATHS = {
    "weights": "bilstm_ru_stream_no_oscar.pth",
    "char2id": "char2id_stream_no_oscar.json"
}

try:
    model_bi, char2id_bi = load_bilstm_autoconfig(BILSTM_PATHS, device=DEVICE)
    print("BiLSTM загружена ✅")
except Exception as e:
    model_bi, char2id_bi = None, None
    print("BiLSTM НЕ загружена ❌:", e)

# sanity: LSTM действительно меняет разрезы
def _pos(restored):
    pos, cur = [], 0
    for ch in restored:
        if ch == ' ': pos.append(cur)
        else: cur += 1
    return set(pos)

sample = "отдамдаромкошку,т.е"
dp_lstm  = restore_spaces_dp_lstm(sample, model=model_bi, char2id=char2id_bi, device=DEVICE)
print("DP+LSTM:", dp_lstm)


[BiLSTM] loaded emb_dim=128, hidden=256, vocab=1017
BiLSTM загружена ✅
DP+LSTM: отдам даром кошку, т. е


In [ ]:
# Массовый прогон DP+LSTM + пост-обработка и запись в task_data


# 1) Пост-обработка: пробел после ключевых слов
def add_space_after_prefixes(
    text: str,
    prefixes = ("ищу", "куплю", "сдаю", "сдам", "продам", "продаю", "сниму"),
    at_start_only: bool = True,
    case_insensitive: bool = True,
    also_bu: bool = False
) -> str:
    """
    Ставит пробел после заданных слов, если сразу далее идёт НЕ пробел/пунктуация.
    - at_start_only=True: применять только в начале строки (обычно для заголовков/запросов).
    - also_bu=True: дополнительно ставить пробел после 'бу' (осторожнее, возможны FP).
    """
    if not text:
        return text

    flags = re.IGNORECASE if case_insensitive else 0
    esc = "|".join(re.escape(p) for p in prefixes)

    if at_start_only:
        # начало строки
        pat = re.compile(rf"(?u)\A({esc})(?=(?!\s)[^\s,.;:!?…()\[\]{{}}\"'—–-])", flags)
    else:
        # на границе слова
        pat = re.compile(rf"(?u)\b({esc})(?=(?!\s)[^\s,.;:!?…()\[\]{{}}\"'—–-])", flags)

    out = pat.sub(lambda m: m.group(1) + " ", text)

    if also_bu:
        bu_pat = re.compile((r"(?ui)\A(бу)(?=(?!\s)[^\s,.;:!?…()\[\]{}\"'—–-])") if at_start_only
                            else (r"(?ui)\b(бу)(?=(?!\s)[^\s,.;:!?…()\[\]{}\"'—–-])"))
        out = bu_pat.sub(r"\1 ", out)

    return out

# 2) Параметры DP+LSTM
ALPHA        = 0.6
MAX_WORD_LEN = 28
HINT_BONUS   = 0.15
USE_LOGIT    = True
TEMP         = 1.4
BIAS         = -0.2
CONF_TH      = 0.6
LAMBDA_CUT   = 0.08
SHORT_PEN    = 1.0
DEVICE       = "cuda" if torch.cuda.is_available() else "cpu"

# 3) Прогон с пост-обработкой
pred_texts = []
pred_texts_post = []

for s in tqdm(task_data["text_no_spaces"].tolist(), desc="Predicting DP+LSTM + postprocess"):
    restored = restore_spaces_dp_lstm(
        s,
        model=model_bi, char2id=char2id_bi, device=DEVICE,
        alpha=ALPHA, max_word_len=MAX_WORD_LEN, hint_bonus=HINT_BONUS,
        use_logit=USE_LOGIT, temp=TEMP, bias=BIAS, conf_threshold=CONF_TH,
        lambda_cut=LAMBDA_CUT, short_penalty=SHORT_PEN
    )
    pred_texts.append(restored)

    # пост-обработка «пробел после ключевых слов»
    restored_post = add_space_after_prefixes(
        restored,
        prefixes=("ищу","куплю","сдаю","сдам","продам","продаю","сниму"),
        at_start_only=True,      # обычно эти слова стоят в начале
        case_insensitive=True,
        also_bu=False            # True, если нужно «бу айфон» -> «бу айфон», но не стоит так как ухудшит скор
    )
    pred_texts_post.append(restored_post)

# 4) Куда класть результат
task_data["restored_text"] = pred_texts_post


# 5) Быстрый просмотр и сохранение
print(task_data[["id", "text_no_spaces", "restored_text"]].head(20))

out_csv = "predicted_text.csv"
task_data[["id", "restored_text"]].to_csv(out_csv, index=False, encoding="utf-8")
print("Saved:", out_csv)

Predicting DP+LSTM + postprocess: 100%|██████████| 1005/1005 [00:17<00:00, 58.95it/s]

    id                    text_no_spaces                         restored_text
0    0                   куплюайфон14про                    куплю айфон 14 про
1    1                ищудомвПодмосковье                 ищу дом в Подмосковье
2    2     сдаюквартирусмебельюитехникой    сдаю квартиру с мебелью и техникой
3    3        новыйдивандоставканедорого         новый диван доставка недорого
4    4                   отдамдаромкошку                     отдам даром кошку
5    5             работавМосквеудаленно              работа в Москве удаленно
6    6             куплютелевизорPhilips               куплю телевизор Philips
7    7           ищугрузчиковдляпереезда            ищу грузчиков для переезда
8    8              ремонтквартирподключ                ремонт квартир подключ
9    9                    куплюноутбукHP                      куплю ноутбук HP
10  10                 ищуквартирууметро                   ищу квартиру уметро
11  11         новаямикроволновкаSamsung           н

In [ ]:
# Индексы пробелов для метрики (сжатые координаты)
def space_positions_compressed(restored_text: str) -> list[int]:
    """
    Возвращает позиции пробелов в 'сжатых' координатах:
    индекс = сколько непробельных символов прошло ДО данного пробела.
    """
    positions = []
    cursor = 0  # считает только непробельные символы
    for ch in restored_text:
        if ch == ' ':
            positions.append(cursor)
        else:
            cursor += 1
    return positions

print("Считаем позиции пробелов для метрики (compressed indices)…")
predicted_positions_list = []
for _, row in tqdm(task_data.iterrows(), total=len(task_data), desc="Finding space positions"):
    restored = row["restored_text"]
    positions = space_positions_compressed(restored)
    predicted_positions_list.append(positions)

# записываем список в DataFrame
task_data["predicted_positions"] = [str(xs) for xs in predicted_positions_list]

# проверка нескольких примеров
print("\nПроверка расчёта позиций:")
for i in range(min(10, len(task_data))):
    original = task_data.loc[i, "text_no_spaces"]
    restored = task_data.loc[i, "restored_text"]
    positions_str = task_data.loc[i, "predicted_positions"]
    print(f"ID: {task_data.loc[i, 'id']}")
    print(f"Original : '{original}'")
    print(f"Restored : '{restored}'")
    print(f"Positions: {positions_str}")
    print("-"*50)

# сохранение в CSV ровно в нужном формате
def create_submission_file(task_data: pd.DataFrame, output_file: str):
    sub = task_data[["id", "predicted_positions"]].copy()
    sub.to_csv(output_file, index=False, encoding="utf-8")
    print(f"Submission file saved to {output_file}")
    print(sub.head())

# создаём финальный submission-файл
create_submission_file(task_data, "task_data.csv")
print("Тип predicted_positions:", type(task_data["predicted_positions"].iloc[0]))


Считаем позиции пробелов для метрики (compressed indices)…


Finding space positions: 100%|██████████| 1005/1005 [00:00<00:00, 12787.71it/s]


Проверка расчёта позиций:
ID: 0
Original : 'куплюайфон14про'
Restored : 'куплю айфон 14 про'
Positions: [5, 10, 12]
--------------------------------------------------
ID: 1
Original : 'ищудомвПодмосковье'
Restored : 'ищу дом в Подмосковье'
Positions: [3, 6, 7]
--------------------------------------------------
ID: 2
Original : 'сдаюквартирусмебельюитехникой'
Restored : 'сдаю квартиру с мебелью и техникой'
Positions: [4, 12, 13, 20, 21]
--------------------------------------------------
ID: 3
Original : 'новыйдивандоставканедорого'
Restored : 'новый диван доставка недорого'
Positions: [5, 10, 18]
--------------------------------------------------
ID: 4
Original : 'отдамдаромкошку'
Restored : 'отдам даром кошку'
Positions: [5, 10]
--------------------------------------------------
ID: 5
Original : 'работавМосквеудаленно'
Restored : 'работа в Москве удаленно'
Positions: [6, 7, 13]
--------------------------------------------------
ID: 6
Original : 'куплютелевизорPhilips'
Restored : 'купл